In [1]:
import torch
import tensorly as tl
from tensorly.decomposition import tensor_train
from Utils import TTSVD,TT_to_tensor
import time
import pandas as pd

In [2]:
tl.set_backend('pytorch')

### Basic Setup of the tensor and the decomposition

In [3]:
d = 4                   # tensor orde
I = 100                # tensor dimension
r = 2                   # choose tt-rank to be constant
rank = [r] * (d - 1)    # tt-rank
shape = [I] * d        # tensor shape

In [4]:
X = torch.randn(shape)

### Compare the speed of the decomposition

In [5]:
time_start = time.time()
X_core = TTSVD(X, rank)
print('MyCode costs:', time.time() - time_start, 's')

time_start = time.time()
factors = tensor_train(X, rank=[1] + rank + [1])
print('Tensorly costs:', time.time() - time_start, 's')

MyCode costs: 9.791947841644287 s
Tensorly costs: 9.547675848007202 s


### Compare the shape of the cores

In [6]:
# Function to add core shapes and datatype to the data dictionary
def add_core_shapes(data, method_name, cores):
    data['Method'].append(method_name)
    for i, core in enumerate(cores):
        core_key_shape = f'Core {i+1} Shape'
        if core_key_shape in data:
            data[core_key_shape].append(core.shape)
        else:
            data[core_key_shape] = [None] * (len(data['Method']) - 1) + [core.shape]
    data['Dtype'].append(cores[0].dtype)

# Prepare data for the table
data = {'Method': [], 'Dtype': []}

# Add core shapes and datatype for MyCode
add_core_shapes(data, 'MyCode', X_core)

# Add core shapes and datatype for Tensorly
add_core_shapes(data, 'Tensorly', factors)

# Create DataFrame
df_cores_dtypes = pd.DataFrame(data)

# Display the table
print(df_cores_dtypes)

     Method          Dtype Core 1 Shape Core 2 Shape Core 3 Shape Core 4 Shape
0    MyCode  torch.float32     (100, 2)  (2, 100, 2)  (2, 100, 2)     (2, 100)
1  Tensorly  torch.float32  (1, 100, 2)  (2, 100, 2)  (2, 100, 2)  (2, 100, 1)


### Compare the Reconstruction error

In [7]:
# Reconstruct the tensor from the decomposed factors using TTSVD
time_start = time.time()
X_reconstructed_mycode = TT_to_tensor(rank,X_core)
print('Reconstruction (MyCode) costs:', time.time() - time_start, 's')

# Reconstruct the tensor from the decomposed factors using tensorly
time_start = time.time()
X_reconstructed_tensorly = tl.tt_to_tensor(factors)
print('Reconstruction (Tensorly) costs:', time.time() - time_start, 's')

# Compute the reconstruction error
error_mycode = torch.norm(X - X_reconstructed_mycode) / torch.norm(X)
error_tensorly = torch.norm(X - X_reconstructed_tensorly) / torch.norm(X)

print('Reconstruction error (MyCode):', error_mycode.item())
print('Reconstruction error (Tensorly):', error_tensorly.item())


Reconstruction (MyCode) costs: 0.08341097831726074 s
Reconstruction (Tensorly) costs: 0.053153038024902344 s
Reconstruction error (MyCode): 0.9999934434890747
Reconstruction error (Tensorly): 0.9999935626983643


reconstruction error not close to zero since the original tensor $X$ is not on manifold with the given rank 